# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [35]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
len(ratings)

100004

In [37]:
# посчитаем количество оценок, выставленных каждым пользователем
count_rating = ratings.groupby('userId').count().add_prefix('total_')
count_rating = count_rating['total_rating'].reset_index()
count_rating.head()

,userId,total_rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [51]:
# оставим в таблице с оценками только тех пользователей, у которых 100 и более оценок
merge_table = ratings.merge(count_rating, on = 'userId', how = 'left')
sort_table = merge_table.query('total_rating >= 100')
sort_table.head()

,userId,movieId,rating,timestamp,total_rating
147,4,10,4.0,949810645,204
148,4,34,5.0,949919556,204
149,4,112,5.0,949810582,204
150,4,141,5.0,949919681,204
151,4,153,4.0,949811346,204


In [45]:
# сгруппируем ро userId и определим мин и макс значение столбца timestamp
min_max = sort_table.groupby('userId').agg({'timestamp':['min','max']})
min_max.head()

timestamp            
               min         max
userId                        
4        949778714   949982274
5       1163373044  1163375145
8       1154389340  1154474527
15       997937239  1469330735
17      1127468587  1127476640

In [53]:
# посчитаем значение LTV
min_max['ltv'] = (min_max['timestamp']['max'] - min_max['timestamp']['min'])
ltv_ = min_max['ltv'].reset_index()
ltv_.head()

,userId,ltv
0,4,203560
1,5,2101
2,8,85187
3,15,471393496
4,17,8053


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов

Необходимо сформировать две таблицы:

1) таблицу с тремя типами выручки для каждого client_id без указания адреса клиента

2) аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [57]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [58]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [59]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [60]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [64]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента
rzd_auto = rzd.merge(auto, how = 'outer', on = 'client_id')
rzd_auto_air = rzd_auto.merge(air, on = 'client_id', how = 'outer')
rzd_auto_air

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [67]:
# аналогичная таблица (как выше) по типам выручки с указанием адреса клиента. Достаточно использовать Left join, т.к.
# в первую таблицу попали все клиенты из базы с адресами
client_base.merge(rzd_auto_air, on = 'client_id', how = 'left')

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:

У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

# Ответ
Для того чтобы связать нужные координаты с фактом покупки необходимо сопоставить дату и время момента покупки с координатами пользователя, которые были зафиксированы приложением в это же время. Тогда мы сможем подтянуть нужные координаты пространства, где находился клиент в момент совершения покупки.
Разброс координат в пространстве на значительное расстояние не повлияет на определение, координат, которые необходимо связаять с покупками, т.к. в конкретный момент времени (момент соверщения покупки) у пользователя могут только какие-то одни координаты, дальнейшие его передвижения не повлияют на определение нужных координат.
Исходя из наборов координат и частоты появления их в списке можно делать предположения о том что точки в пространстве, которые встречаются часто - это место жительства клиента, либо его работа и можно предложить клиенту самому указать, что это такое - дом или работа, по аналогии как в гугл картах. Зная это, можно рекомедновать точки продаж рядом с домом либо рядом с работой клиента.
Также зная текущее местополежние клиента, можно показать в радиусе N-километров точки продаж, если это продажа в офлайн магазине.
По перемещениям клиента можно понимать по каким районам города перемещается клиент, где проходят его стандартные маршруты, по каким магазин он ходит. Исходя из этого можно формировать картину потребителя для верных рекомендаций.
Если клиент, согласно координат, совершает поездки в другие города, то можно расширять географию точек, которые предлагать клиенту и не ограничиваться городом его постоянной дислокации, предлагая пункты продаж и в других городах.

Также для целей аналитики, имея данные по координатам можно в магазинах, районах города, городах продажи идут лучше, чем в дргуих, а далее анализировать возможные причины этого.